In [1]:
%bookmark HOME "/home/jupyter/meta_XLM/XLM" 
%cd -b HOME

(bookmark:HOME) -> /home/jupyter/meta_XLM/XLM
/home/jupyter/meta_XLM/XLM


In [3]:
%env csv_path=/home/jupyter
%env output_dir=/home/jupyter/data
%env data_type=para
%env languages=Limbum,Ngiemboon
! python ../bible.py --csv_path $csv_path --output_dir $output_dir --data_type $data_type --languages $languages

env: csv_path=/home/jupyter/csvs
env: output_dir=/home/jupyter/data
env: data_type=para
env: languages=Limbum,Ngiemboon
Limbum - Ngiemboon
======= Read 7950 totals samples
======= Delete 50 samples
======= Save 7900 samples


In [4]:
%env PARA=True          
%env MONO=True       
                   
%env PARA_PATH=/home/jupyter/data/para/Limbum-Ngiemboon     
%env MONO_PATH=""  
%env SAME_VOCAB=True    
%env nCodes=10000             
%env shuf_n_samples=1000000   
%env threads_for_tokenizer=16 
%env test_size=10             
%env val_size=10              

# tools paths
%env TOKENIZE=tools/tokenizer_our.sh
%env LOWER_REMOVE_ACCENT=tools/lowercase_and_remove_accent.py
%env FASTBPE=tools/fastBPE/fast


%env OUTPATH=/home/jupyter/models/africa/cluster3/data/Limbum_Ngiemboon/processed
# create output path
! mkdir -p $OUTPATH

! chmod +x $FASTBPE
! chmod +x ../build_meta_data.sh
! chmod +x tools/mosesdecoder/scripts/tokenizer/*.perl

%env n_samples=-1

%env sub_task=Limbum-Ngiemboon:-1

env: PARA=True
env: MONO=True
env: PARA_PATH=/home/jupyter/data/para/Limbum-Ngiemboon
env: MONO_PATH=""
env: SAME_VOCAB=True
env: nCodes=10000
env: shuf_n_samples=1000000
env: threads_for_tokenizer=16
env: test_size=10
env: val_size=10
env: TOKENIZE=tools/tokenizer_our.sh
env: LOWER_REMOVE_ACCENT=tools/lowercase_and_remove_accent.py
env: FASTBPE=tools/fastBPE/fast
env: OUTPATH=/home/jupyter/models/africa/cluster3/data/Limbum_Ngiemboon/processed
env: n_samples=-1
env: sub_task=Limbum-Ngiemboon:-1


In [5]:
# moins de codes que les autres, sinon "fast: fastBPE/fastBPE.hpp:458: void fastBPE::readCodes(const char* ...).Assertion `codes.find(pair) == codes.end()' failed."
%env nCodes=7000 
%env add_para_data_to_mono_data=False
! ../build_meta_data.sh $sub_task $n_samples $add_para_data_to_mono_data

env: nCodes=7000
env: add_para_data_to_mono_data=False
params ok !
*** Cleaning and tokenizing Limbum-Ngiemboon data ... ***
Tokenizer Version 1.1
Language: Limbum
Number of threads: 8
*** Tokenized (+ lowercase + accent-removal) Limbum-Ngiemboon.Limbum data to /home/jupyter/data/para/Limbum-Ngiemboon/? ***
Tokenizer Version 1.1
Language: Ngiemboon
Number of threads: 8
*** Tokenized (+ lowercase + accent-removal) Limbum-Ngiemboon.Ngiemboon data to /home/jupyter/data/para/Limbum-Ngiemboon/? ***


*** split into train / valid / test ***
shuf: write error: Broken pipe
shuf: write error
shuf: write error: Broken pipe
shuf: write error
shuf: write error: Broken pipe
shuf: write error
shuf: write error: Broken pipe
shuf: write error



***build the training set for BPE tokenization (7000 codes)***


***shuf ... Generating 1000000 random permutations of training data and store result in /home/jupyter/models/africa/cluster3/data/Limbum_Ngiemboon/processed/Limbum-Ngiemboon/bpe.train***


***Lea

In [6]:
%env OUTPATH=/home/jupyter/models/africa/cluster3/data/Limbum_Ngiemboon/processed
%env exp_id=maml
# If you don't have enough RAM or swap memory, leave these three parameters to True, otherwise you may get an error like this when evaluating 
# RuntimeError: copy_if failed to synchronize: cudaErrorAssert: device-side assert triggered
%env remove_long_sentences_train=True
%env remove_long_sentences_valid=True
%env remove_long_sentences_test=True
#--remove_long_sentences_train $remove_long_sentences_train --remove_long_sentences_valid $remove_long_sentences_valid --remove_long_sentences_test $remove_long_sentences_test

env: OUTPATH=/home/jupyter/models/africa/cluster3/data/Limbum_Ngiemboon/processed
env: exp_id=maml
env: remove_long_sentences_train=True
env: remove_long_sentences_valid=True
env: remove_long_sentences_test=True


In [8]:
# le plus grand fichier à tensor_len phrases, on cherche le multiple de "batch_size" le plus proche de ce 
# nombre par valeur supérieur : epoch_size doit etre un multiple non nul de ce nombre (pour ne pas gaspiller) 

def getEpochSize(tensor_len, batch_size):
    i = tensor_len
    while True :
        if i%batch_size == 0 :
            return i//batch_size
        i = i + 1

import io

def n_lines(file_path):
    return len(io.open(file_path, encoding='UTF-8').read().split('\n'))

In [9]:
tensor_len = n_lines(file_path = "/home/jupyter/models/africa/cluster3/data/Limbum_Ngiemboon/processed/Limbum-Ngiemboon.Limbum.train")
print("tensor_len = " + str(tensor_len))

tensor_len = 6322


In [10]:
%env epoch_size=6322

%env lgs=Limbum-Ngiemboon

# limit the number of examples (-1 by default for non limitation)
%env train_n_samples=-1
%env valid_n_samples=-1
%env test_n_samples=-1
#--train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples

# stopping criterion (if criterion does not improve 10 times)
%env stopping_criterion=_valid_mlm_ppl,10
%env eval_bleu false

env: epoch_size=6322
env: lgs=Limbum-Ngiemboon
env: train_n_samples=-1
env: valid_n_samples=-1
env: test_n_samples=-1
env: stopping_criterion=_valid_mlm_ppl,10
env: eval_bleu=false


In [ ]:
%env mlm_steps=Limbum,Ngiemboon,Limbum-Ngiemboon
%env batch_size=8
%env max_epoch=100
%env dump_path=/home/jupyter/models/africa/cluster3
! python train.py --exp_name mlm_tlm_LimbumNgiemboon --exp_id $exp_id --dump_path $dump_path --data_path $OUTPATH --lgs $lgs --clm_steps '' --mlm_steps $mlm_steps --emb_dim 1024 --n_layers 6 --n_heads 8 --dropout 0.1 --attention_dropout 0.1 --gelu_activation true --batch_size $batch_size --bptt 256 --optimizer adam,lr=0.0001 --epoch_size $epoch_size --max_epoch $max_epoch --validation_metrics _valid_mlm_ppl --stopping_criterion $stopping_criterion --eval_bleu $eval_bleu --remove_long_sentences_train $remove_long_sentences_train --remove_long_sentences_valid $remove_long_sentences_valid --remove_long_sentences_test $remove_long_sentences_test --train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples

env: mlm_steps=Limbum,Ngiemboon,Limbum-Ngiemboon
env: batch_size=8
env: max_epoch=100
env: dump_path=/home/jupyter/models/africa/cluster3
FAISS library was not found.
FAISS not available. Switching to standard nearest neighbors search implementation.
SLURM job: False
0 - Number of nodes: 1
0 - Node ID        : 0
0 - Local rank     : 0
0 - Global rank    : 0
0 - World size     : 1
0 - GPUs per node  : 1
0 - Master         : True
0 - Multi-node     : False
0 - Multi-GPU      : False
0 - Hostname       : african-translator-vm-bis-vm
INFO - 05/20/20 09:54:32 - 0:00:00 - ============ Initialized logger ============
INFO - 05/20/20 09:54:32 - 0:00:00 - accumulate_gradients: 1
                                     ae_steps: []
                                     amp: -1
                                     asm: False
                                     attention_dropout: 0.1
                                     batch_size: 8
                                     beam_size: 1
                 

In [ ]:
# ! rsync -av --progress /content/meta_XLM/XLM/dumped/mlm_tlm_BafiaBulu  /content/drive/"My Drive"/African_Translator/models/africa/cluster1

**Train a (unsupervised/supervised) MT from a pretrained meta-model**

In [18]:
%env epoch_size=6360
%env lgs=Bafia-Bulu

env: epoch_size=6360
env: lgs=Bafia-Bulu


In [19]:
# limit the number of examples (-1 by default for non limitation)
%env train_n_samples=-1
%env valid_n_samples=-1
%env test_n_samples=-1
#--train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples

%env eval_bleu=true
! chmod +x src/evaluation/multi-bleu.perl

env: train_n_samples=-1
env: valid_n_samples=-1
env: test_n_samples=-1
env: eval_bleu=true


In [24]:
%env stopping_criterion=valid_mt_bleu,10
%env validation_metrics=valid_mt_bleu
%env reload_model=dumped/mlm_tlm_BafiaBulu/maml/best-valid_mlm_ppl.pth,dumped/mlm_tlm_BafiaBulu/maml/best-valid_mlm_ppl.pth
%env ae_steps=Bafia,Bulu
%env bt_steps=Bafia-Bulu-Bafia,Bulu-Bafia-Bulu

env: stopping_criterion=valid_mt_bleu,10
env: validation_metrics=valid_mt_bleu
env: reload_model=dumped/mlm_tlm_BafiaBulu/maml/best-valid_mlm_ppl.pth,dumped/mlm_tlm_BafiaBulu/maml/best-valid_mlm_ppl.pth
env: ae_steps=Bafia,Bulu
env: bt_steps=Bafia-Bulu-Bafia,Bulu-Bafia-Bulu


In [ ]:
%env train_n_samples=-1
%env valid_n_samples=-1
%env test_n_samples=-1

%env max_epoch=100

# unsupervised MT
! python train.py --exp_name UnSupMT_BafiaBulu --exp_id $exp_id --dump_path ./dumped/ --reload_model $reload_model --data_path $OUTPATH --lgs $lgs --ae_steps $ae_steps --bt_steps $bt_steps --word_shuffle 3 --word_dropout 0.1 --word_blank 0.1 --lambda_ae '0:1,100000:0.1,300000:0' --encoder_only false --emb_dim 1024 --n_layers 6 --n_heads 8 --dropout 0.1 --attention_dropout 0.1 --gelu_activation true --tokens_per_batch 2000 --batch_size $batch_size --bptt 256 --optimizer adam_inverse_sqrt,beta1=0.9,beta2=0.98,lr=0.0001 --epoch_size $epoch_size --max_epoch $max_epoch --eval_bleu $eval_bleu --stopping_criterion $stopping_criterion --validation_metrics $validation_metrics --train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples --remove_long_sentences_train $remove_long_sentences_train --remove_long_sentences_valid $remove_long_sentences_valid --remove_long_sentences_test $remove_long_sentences_test    

In [ ]:
#! rsync -av --progress /content/meta_XLM/XLM/dumped/mlm_tlm_BafiaBulu  /content/drive/"My Drive"/African_Translator/models/africa/cluster1

In [ ]:
%env train_n_samples=-1
%env valid_n_samples=-1
%env test_n_samples=-1

%env max_epoch=100
%env batch_size=1
# supervised MT
%env mt_steps=Bafia-Bulu,Bulu-Bafia          
! python train.py --exp_name MT_BafiaBulu --exp_id $exp_id  --dump_path ./dumped/ --reload_model $reload_model --data_path $OUTPATH --lgs $lgs --ae_steps $ae_steps --mt_steps $mt_steps --bt_steps $bt_steps --word_shuffle 3 --word_dropout 0.1 --word_blank 0.1 --lambda_ae '0:1,100000:0.1,300000:0' --encoder_only false --emb_dim 1024 --n_layers 6 --n_heads 8 --dropout 0.1 --attention_dropout 0.1 --gelu_activation true --tokens_per_batch 2000 --batch_size $batch_size --bptt 256 --optimizer adam_inverse_sqrt,beta1=0.9,beta2=0.98,lr=0.0001 --epoch_size $epoch_size --max_epoch $max_epoch --eval_bleu $eval_bleu --stopping_criterion $stopping_criterion --validation_metrics $validation_metrics --train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples --remove_long_sentences_train $remove_long_sentences_train --remove_long_sentences_valid $remove_long_sentences_valid --remove_long_sentences_test $remove_long_sentences_test    

In [ ]:
#! rsync -av --progress /content/meta_XLM/XLM/dumped/mlm_tlm_BafiaBulu  /content/drive/"My Drive"/African_Translator/models/africa/cluster1

In [ ]:
d=[]
while(1):
  d.append(d)